# Calculation of the entropy for sources with and without memory
## Introduction
This tutorial will get you familiar with the calculation of the entropy associated with a given source. We start by recalling some definitions and fundamental results from the [Shannon's information theory](http://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf). We will then move to consider some practical examples, focusing on sources generating grey scale natural images (given that the whole training is about image and video).

## Preliminary remarks
Consider a discrete source of information $S$ which emits symbols $a_i$ from an *alphabet* $A$ and according to a Probability Mass Function (PMF) $p_S$:
$$
\large
S = \{A, p_S\},
$$

where, $A$ is the set of symbols $\{a_0, a_1,\ldots,a_N\}$ and $p_S: A \rightarrow [0, 1]$. In other words, the source $S$ emits symbols from $A$ and the $i$-th symbol $a_i$ has a probability of being emitted $p_S(a_i) = p_i$. The number of symbols emitted by a source characterises its arity: sources emitting two symbols are *binary*, three symbols *ternary*, $n$ symbols *n-ary* and so on. For each symbol $a_i\in A$, its **information content** ($I(a_i)$) is defined as:
$$
\large
I(a_i) = \log_2\left(\frac{1}{p_S(s_i)}\right),\quad[bits]
$$

This information content can be thought as the minimum amount of bits required to encode $a_i$. Note that symbols less likely to appear will require more bits and viceversa. From the information content, the entropy of source $S$ is defined as the *expected* information content:
$$
\large
H(S) = E[I(a_i)] = \sum_{i=0}^N -p_S(a_i)\cdot\log_2(p_S(a_i))\quad[bits/symbol]
$$

The entropy measures the *average* number of bits needed to code the symbols emitted by $S$. The Shannon's noiseless source coding theorem, tells you that such an *average* number of bits is also the *lowest* possibly achievable by any coding scheme one can come up with. Another consideration that follows from the definition of entropy is the fact that source whose PMF is uniform will lead to an entropy equal to $\log_2(|A|)$, where $|\cdot|$ denotes the set's cardinality.

The source $S$ considered so far is said to be memoryless since each symbol is emitted irrespective to the symbols emitted earlier. Conversely, sources emitting symbols based on their previous output are said to be *with memory*. Images can be considered as instances emitted by a source with memory. In fact, the intensity value of adjacent pixels is likely to be correlated and indeed simple models (e.g. first order Auto-Regressive (AR(1)) stochastic processes) are often used to study the performance limit of some coding tools such as frequency transforms (e.g. by computing the transform gain). If a source has memory, then the amount of information needed to transmit a given symbol can be reduced based on the symbols already transmitted. Recalling again the example of images, knowing the value of pixels located at even positions (in raster scan order) can help to infer the value of the adjacent pixels located at the odd positions: this is the fundamental principle behind *predictive coding*. For sources with memory, one can compute the entropy associated with the transmission of symbols in light of what has been done in the past. To accomplish this task we will use the concept of *conditional entropy*. Consider two sources of information $X$ and $Y$ with conditional probability mass function $p_{Y|X}$, the conditional entropy $H(Y|X)$ can be computed as:
$$
\large
H(Y|X) = -\sum_{i,j}p(x_i,y_j)\cdot log_2(p_{Y|X}(y_i|x_j))\quad[bits/symbol],
$$
where $p(x_i, y_i)$ denotes the *joint* probability. We now have all ingredients ready to write some toy examples and put *the theory in practice*.


## Example 1: Calculation of the entropy of a memoryless source
As mentioned at the beginning of this tutorial, we will consider a particular source of information that generates grey scale natural images whereas each pixel is represented with 8 bits per pixel \[bpp\]. In particular, we will compute the entropy of such a source assuming that among the many images generated, the well known [*cameraman.tif*](https://homepages.cae.wisc.edu/~ece533/images/cameraman.tif) sample is one example. This will allow us to estimate the source's PMF which is the key ingredient to compute the Shannon's entropy. We start by loading the required Python packages (i.e. `cv2` and `numpy`) and then read the image in memory.

In [ ]:
import cv2
import numpy as np
image = cv2.imread('../input-data/cameraman.tif', cv2.IMREAD_UNCHANGED)

We can visualise the image by using the function `imshow` from the the `matplotlib` package.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(7, 7))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title(f"Cameraman grey scale image with size {image.shape[0]}x{image.shape[1]}");

As mentioned above, we'll now estimate the source's PMF by computing the normalised frequency of each one of the 256 grey levels from the image's pixels. To do so we will use the `histogram` function from `numpy` which returns the normalised frequency of each one of the 256 gray levels over all images pixels. More information about this function is available [here](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html)

In [ ]:
p, bins = np.histogram(image, 256, [0, 255], density=True)
plt.bar(range(len(p)), p)
plt.xlabel('Gray level value', fontsize=16)
plt.ylabel('Normalised frequency', fontsize=16)
plt.grid()
plt.title('Normalised histogram for the cameraman.tif image', fontsize=16);

We note from the code cell above that the function `histogram` from `numpy` has the parameter `density` set to true. This will tell the function to return the normalised histogram. If not set, it will return the absolute histogram instead. We also note from the graph obtained above that the histogram is *bi-modal*, i.e. it has two peaks: these are associated with the man's black coat and the grey background. This result should also immediately suggest that the Shannon's entropy won't equate to 8 bits per pixels as the image pixels are currently represented. In other words, we can surely find a code which will require less than 8 bpp to represent this image. The question is now how many bits do we then need? The answer will be given by the computation of the entropy.

In [ ]:
index_valid = np.where(p != 0)
H = -np.sum(np.multiply(p[index_valid], np.log2(p[index_valid])))
print(f"Estimated entropy for souce generating gray scale images: {H:.2f} bits per pixel [bpp]")

We first note that the value of the entropy confirms our observation above, i.e. that more bits can be squeezed to represent the image pixels given the bi-modal nature of the PMF estimated. Please also note the emphasis on the word *estimated*: we are estimating the PMF of our source from one instance (i.e. the *cameraman.tif* image), which we've assumed to be a fairly accurate representative of the data generated. In practice, a more accurate estimate of the PMF would be obtained, had more images been available.

In terms of the code used to compute the entropy, it is worth noting the use of `where` from `numpy` which returns an array with elements from $p$ where the boolean condition is `true`. Such check is required to purge from $p$ all those gray levels associated with zero frequency, since for them the logarithm operation is undefined.

## Example 2: Calculation of the entropy for a source with memory
We now want to go a step further and ask ourselves whether we can reduce more the number of bits required to represent image pixels by considering some sort of data correlation. We remember that if two sources $X$ and $Y$ are correlated, we can reduce the number of bits required to transmit (say) $Y$ by knowing $X$. More precisely, we can transmit the residual information associated with $Y$ which cannot be inferred from $X$. In this example, we'll consider again our initial source which generates grey scale images such as the *cameraman.tif*. To look for data correlation, let's derive from the original source two additional ones as follows:
 * Source $X$ is the source generating all values of pixels located at even indexes in a raster scan order.
 * Source $Y$ is the source generating all values of pixels located at odd indexes in a raster scan order.

Our original information source is generating grey scale *natural* images so one would expect that the value of pixels lying on even indexes is quite correlated with that of its adjacent odd indexes counterpart. To provide more evidence to such statement, we can plot a scatter diagram of a limited set of values for $X$ and $Y$ extracted from *cameraman.tif*.

In [ ]:
image = image.flatten()
X = image[:image.size-1:2]
Y = image[1:image.size:2]
plt.plot(X[0:1000], Y[0:1000], 'o')
plt.xlabel('Pixel values for source X', fontsize=16)
plt.ylabel('Pixel values for source Y', fontsize=16)
plt.grid()

As may be noted, pairs of values from $X$ and $Y$ are aligned along a 45 degree straight line. Pairs would have been spread throughout the whole 2D scatter plot, had the image been associated with random noise (i.e. each pixel value is uncorrelated with the others). We notice few outliers in the scatter plot: these are associated with pixels lying at the image edges where a sudden transition of the values is likely to happen.

To measure how much information one needs to transmit for source $Y$ assuming that $X$ is somewhat known, we need to compute the conditional entropy $H(Y|X)$. We remember that the formula is as follows:

$$
\large
H(Y|X) = -\sum_{i,j}p(x_i,y_j)\cdot log_2(p_{Y|X}(y_i|x_j))
$$

From the formula above we note that we need two ingredients: the joint probability mass function $p(X,Y)$ and the conditional one $p(Y|X)$. The joint probability can easily be computed by considering all possible values source $X$ and $Y$ can emit: $[0, 255]$ in our case. Then, for each pair of values $(x_i, y_j)$ we can count how many times source $X$ has emitted value $x_i$ and source $Y$ value $y_j$. Accordingly, we declare a 2D array with size $256\times 256$ and initialise all values to zero. Then we jointly loop through all pixel values associated with $X$ and $Y$ and increment the cell in such a 2D array indexed by the value of the pair $(x_i, y_j)$. The following code cell implements this processing.

In [ ]:
p_joint = np.zeros((256, 256))
for i in range(len(X)):
    p_joint[X[i], Y[i]] += 1
    
p_joint = np.divide(p_joint, len(X))

The last statement in the previous code cell normalises the frequency of all pairs so to have an estimate of the joint pdf. To compute the conditional probability, we remember the formula from statistics at Uni:

$$
\large
P(Y|X) = \frac{P(X,Y)}{P(X)}
$$

where $P(X)$ is the marginal probability for source $X$. Given the joint probability $P(X,Y)$, the marginal for $X$ is simply the sum of all $p(x_i,y_j)$ where the value $x_i$ is fixed. We note that this corresponds to add up all elements from $P(X,Y)$ along a given row ($X$ represents the rows in the 2D array of joint probability we just built). Adding elements along a given direction in a 2D array is easily obtained via function `sum` from `numpy`.

In [ ]:
p_marginal = np.sum(p_joint, 1)
p_marginal_ext = np.tile(p_marginal, (256, 1))

We note from the code above that the `sum` function will return a 1D array with 256 cells. However, from the definition of conditional probability given above, we need to divide each row of $P(X,Y)$ by the corresponding cell in $P(X)$ (i.e. the 1D array). This can be done using a for loop (not very efficient) or by element wise division where the 1D array associated with $P(X)$ is extended to a 2D using the function `tile` from `numpy` which simply extents an array along a given dimension. The conditional probability $P(Y|X)$ can now be computed as shown in the following code cell.

In [ ]:
index_valid = np.where(p_marginal_ext != 0)
p_cond = np.zeros((256, 256))
p_cond[index_valid] = np.divide(p_joint[index_valid], p_marginal_ext[index_valid])

We can now compute the conditional entropy, using the formula we recalled in the preliminary remarks section:

In [ ]:
index_valid = np.where(p_cond != 0)
H_cond = -np.sum(np.multiply(p_joint[index_valid], np.log2(p_cond[index_valid])))
print(f"Conditional entropy value H(Y|X): {H_cond:.2f} bpp")

We can now finally appreciate the reduction of bits required to transmit pixels generated by source $Y$ if the value for pixels generated by source $X$ is known.

## Concluding remarks
We shall wrap up this tutorial with some thoughts and considerations. The first one is on the last example, in particular on how one could realised a more practical coding scheme which jointly encodes the pixels values of pixels. A possible workflow could be as follows:
 * By scanning the image pixels in raster scan order, consider all pairs of pixels at even and odd indexes and compute their joint histogram (i.e. use the values of the 2D array `p_joint` without dividing them by the total of pairs considered).
 * Derive a Huffman code for these pairs where the most likely to appear pairs will be coded with shorter codewords. You can implement your own version of the Huffman's algorithm or use any of the Python's packages out there (e.g. the [Huffman](https://pypi.org/project/huffman/) one from `pypi`).
 * Write the Huffman table in the bitstream along with codeword associated with each pair of pixels. The decoder would simply read the Huffman table and then start parsing the bitstream: whenever a codeword is matched, the corresponding pair of pixel values would be written in the buffer associated with the decoded image

The second remark worth sharing is about the value of the entropy associated with the *cameraman.tif* image. It was 7.03 bits per pixel. Despite this being less than 8, any coding scheme using an integer number of bits (Huffman coding is one of those) would still use 8 bits anyway. A coding scheme such as arithmetic coding could archieve such a theoretical limit at the cost of increased complexity.

As a third remark, we shall observe that this tutorial was about the fundamentals results of the Shannon's information theory, thus we focused on entropy and its conditional variant which gave us the tip that for images, where a degree of spatial correlation exists among pixels, better coding schemes can be devised. We didn't appraised another classical and simple coding scheme: ***run length encoding***. We should expected that also this one could do a good job in reducing the bits pixels.

Finally, on a more general level, we also observe that some of the considerations made here also hold for different types of signal. As an example audio samples show a high degree of correlation along the temporal dimension. Accordingly, we would expect that the conditional entropy will yield to lower values than the entropy of the memoreless version of the audio source.